# Exception Handling in Elastic-Script

This notebook demonstrates the comprehensive exception handling capabilities in elastic-script.

## Features

- **TRY/CATCH/FINALLY** - Standard exception handling blocks
- **Named Exceptions** - Catch specific exception types like `http_error`, `timeout_error`
- **@error Variable** - Structured error information accessible in CATCH blocks
- **THROW/RAISE** - Throw custom exceptions with messages and error codes

## Exception Types

| Type | Description |
|------|-------------|
| `error` | Generic error (catch-all) |
| `http_error` | HTTP/network errors |
| `timeout_error` | Timeout errors |
| `division_error` | Division by zero |
| `null_reference_error` | Null pointer/reference errors |
| `type_error` | Type mismatch errors |
| `validation_error` | Validation failures |
| `not_found_error` | Resource not found |
| `permission_error` | Permission/authorization errors |
| `esql_error` | ES|QL query errors |
| `function_error` | Built-in function errors |

## 1. Basic TRY/CATCH

Catch and handle errors gracefully:

In [ ]:
%%elastic_script

CREATE PROCEDURE basic_try_catch()
BEGIN
    TRY
        PRINT 'Attempting risky operation...';
        THROW 'Something went wrong!';
        PRINT 'This will not print';
    CATCH
        PRINT 'Caught error: ' || error['message'];
    END TRY
    
    PRINT 'Continuing after error handling';
END PROCEDURE

CALL basic_try_catch();

## 2. The @error Variable

In a CATCH block, the `error` variable (accessed via `@error` in syntax) contains structured error information:

In [ ]:
%%elastic_script

CREATE PROCEDURE error_details()
BEGIN
    TRY
        THROW 'File not found' WITH CODE 'FILE_404';
    CATCH
        PRINT 'Error Details:';
        PRINT '  Message: ' || error['message'];
        PRINT '  Code: ' || error['code'];
        PRINT '  Type: ' || error['type'];
    END TRY
END PROCEDURE

CALL error_details();

## 3. THROW with Error Codes

Use `THROW message WITH CODE code` to include an error code:

In [ ]:
%%elastic_script

CREATE PROCEDURE throw_with_code(status NUMBER)
BEGIN
    TRY
        IF status == 404 THEN
            THROW 'Resource not found' WITH CODE 'HTTP_404';
        ELSEIF status == 500 THEN
            THROW 'Internal server error' WITH CODE 'HTTP_500';
        ELSEIF status == 401 THEN
            THROW 'Unauthorized' WITH CODE 'HTTP_401';
        ELSE
            PRINT 'Status OK: ' || status;
        END IF
    CATCH
        PRINT 'HTTP Error [' || error['code'] || ']: ' || error['message'];
    END TRY
END PROCEDURE

CALL throw_with_code(404);
CALL throw_with_code(500);
CALL throw_with_code(200);

## 4. FINALLY Block

The FINALLY block always executes, whether an exception occurred or not. Use it for cleanup:

In [ ]:
%%elastic_script

CREATE PROCEDURE with_finally(should_fail BOOLEAN)
BEGIN
    PRINT 'Starting operation (should_fail=' || should_fail || ')';
    
    TRY
        PRINT '  Acquiring resources...';
        
        IF should_fail THEN
            THROW 'Operation failed!';
        END IF
        
        PRINT '  Operation succeeded!';
    CATCH
        PRINT '  Handling error: ' || error['message'];
    FINALLY
        PRINT '  Releasing resources (cleanup)';
    END TRY
    
    PRINT 'Done.';
END PROCEDURE

CALL with_finally(false);
PRINT '---';
CALL with_finally(true);

## 5. RAISE as Alias for THROW

`RAISE` is an alias for `THROW` - use whichever you prefer:

In [ ]:
%%elastic_script

CREATE PROCEDURE raise_example()
BEGIN
    TRY
        RAISE 'Using RAISE instead of THROW' WITH CODE 'CUSTOM_001';
    CATCH
        PRINT 'Caught: ' || error['message'];
        PRINT 'Code: ' || error['code'];
    END TRY
END PROCEDURE

CALL raise_example();

## 6. THROW with Expressions

Both the message and code can be expressions (not just string literals):

In [ ]:
%%elastic_script

CREATE PROCEDURE throw_expressions(item_id STRING)
BEGIN
    DECLARE error_msg STRING = 'Item not found: ' || item_id;
    DECLARE error_code STRING = 'ITEM_' || item_id || '_NOT_FOUND';
    
    TRY
        THROW error_msg WITH CODE error_code;
    CATCH
        PRINT 'Error: ' || error['message'];
        PRINT 'Code: ' || error['code'];
    END TRY
END PROCEDURE

CALL throw_expressions('ABC123');

## 7. Nested TRY/CATCH

TRY/CATCH blocks can be nested for fine-grained error handling:

In [ ]:
%%elastic_script

CREATE PROCEDURE nested_try_catch()
BEGIN
    TRY
        PRINT 'Outer TRY block';
        
        TRY
            PRINT '  Inner TRY block';
            THROW 'Inner error';
        CATCH
            PRINT '  Inner CATCH: ' || error['message'];
            -- Re-throw a different error
            THROW 'Outer error from inner catch';
        END TRY
        
        PRINT 'This will not print';
    CATCH
        PRINT 'Outer CATCH: ' || error['message'];
    END TRY
END PROCEDURE

CALL nested_try_catch();

## 8. Practical Example: Resilient Data Processing

Use exception handling for robust data processing:

In [ ]:
%%elastic_script

CREATE PROCEDURE process_items(items ARRAY)
BEGIN
    DECLARE processed NUMBER = 0;
    DECLARE failed NUMBER = 0;
    
    FOR item IN items LOOP
        TRY
            -- Simulate processing that might fail
            IF item['value'] < 0 THEN
                THROW 'Negative value not allowed' WITH CODE 'VALIDATION_ERROR';
            END IF
            
            PRINT 'Processed: ' || item['name'] || ' = ' || item['value'];
            SET processed = processed + 1;
        CATCH
            PRINT 'Failed: ' || item['name'] || ' - ' || error['message'];
            SET failed = failed + 1;
        END TRY
    END LOOP
    
    PRINT '---';
    PRINT 'Summary: ' || processed || ' processed, ' || failed || ' failed';
END PROCEDURE

CALL process_items([
    {'name': 'item1', 'value': 100},
    {'name': 'item2', 'value': -50},
    {'name': 'item3', 'value': 200},
    {'name': 'item4', 'value': -10},
    {'name': 'item5', 'value': 300}
]);

## Summary

Exception handling in elastic-script provides:

1. **TRY/CATCH/FINALLY** - Standard structured exception handling
2. **@error variable** - Rich error information with message, code, type, and stack_trace
3. **THROW/RAISE** - Custom exception throwing with optional error codes
4. **Expression support** - Dynamic error messages and codes
5. **Nested handling** - Fine-grained control over error handling scope

### Best Practices

- Use FINALLY for cleanup (closing connections, releasing resources)
- Include error codes for programmatic error handling
- Log error details in CATCH blocks for debugging
- Keep TRY blocks focused on the risky operation
- Re-throw with context when appropriate